# Analysis of Children's motion data

## 1. Import data

In [ ]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops
from skimage.color import label2rgb
import scipy.io
import pprint
import operator
from matplotlib.pyplot import figure
import glob
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests
import datetime
import pydicom

In [ ]:
from functions import *

In [ ]:
# define output folder:
folder = '/pc_disk1/moco/StudentProjects/MSc/Hannah/FrontRadio_Eichhorn_2021/analysis/'

# get all subjects:
GA = ['2017-03-31_ZA', '2017-04-07_SK', '2017-06-02_MN', '2017-06-16_SJAS', '2017-06-23_AKDW', '2017-07-07_WHO', '2017-11-24_JA', '2018-05-25_MT', '2019-01-11_MS_0607175918', '2015-04-14_EB', '2015-05-19_EV', '2016-09-30_AMRH', '2016-10-21_JNT', '2016-12-09_MN', '2017-02-03_PL', '2017-03-10_SJAS', '2017-03-17_MTD', '2017-03-28_NPN']
no_GA = ['2015-09-01_TSS', '2015-10-06_RMI', '2015-10-09_SK', '2015-12-01_FCA', '2015-12-01_VTB', '2015-12-15_LR', '2015-12-15_PN', '2015-12-22_SRo', '2016-02-26_MR', '2016-03-22_NLL', '2016-04-05_ESS', '2016-04-05_JMPD', '2016-04-26_JJ', '2016-05-20_NLL', '2016-05-31_BNP', '2016-06-21_AHA', '2016-06-28_MS', '2016-06-28_NLVS', '2016-08-05_EJ', '2016-09-30_VJ', '2016-10-04_JMPD', '2016-10-11_JJ', '2016-12-13_OKR', '2017-01-06_NM', '2017-01-17_JVJ', '2017-02-14_SLA', '2017-03-14_ESS', '2017-06-06_MS', '2017-06-06_RL', '2017-06-13_LR', '2017-06-20_OH', '2017-09-05_NSH', '2017-09-05_SE2', '2017-09-12_ICV', '2017-09-26_MS', '2017-10-03_RA', '2017-10-10_MA', '2018-01-02_ACK', '2018-01-09_NSN', '2018-03-16_EA', '2018-03-16_TP', '2018-04-17_MWA', '2018-04-24_LR']
subjs = np.append(no_GA, GA)
print(subjs)

In [ ]:
# get age of GA subjects:
file1 = open(folder+"GA/age.txt","r") 
ages = file1.read()
ageslist = ages[:-1].split("\n")
print(ageslist)
file1.close()

In [ ]:
Scan_Times = np.loadtxt(folder+'Scan_Times.txt', dtype=str, delimiter=', ', skiprows=1)

In [ ]:
'''#save to file GA info / point cloud centroid added to centroids 
for sub in GA:
    print(sub)
    # reads the vox2ras for the corresponding subject
    vox2ras = np.array(np.loadtxt(folder+"v2r/"+sub+".txt", dtype=float)).reshape(4,4)
    c_,r,_ = getCentriods(folder+"regions binarization/"+sub,vox2ras)
    pcl_centr = PClCentroid(sub)
    print('PointCloud centroid: ', pcl_centr)
    c = np.append(c_, [pcl_centr], axis=0)
    np.savetxt(folder+'GA/centroids '+sub+'.out', np.array(c))
    mat_files = StartEndFrameNr([sub], Scan_Times, folder+'GA/')
    h = computeHistograms2(mat_files,c)
    np.savetxt(folder+'GA/histogramREF '+sub+'.out', np.array(h))'''

In [ ]:
# load GA centroids and motion data ('histogram'):
GA_centrFile = []
GA_maxFile = []
GA_avgFile = []
GA_medianFile = []
GA_patients = []
GA_motionfreeFile = []
idx = 0
idxs = []
hGA = []
for sub in GA:
    print(sub)
    vox2ras = np.array(np.loadtxt(folder+"v2r/"+sub+".txt", dtype=float)).reshape(4,4)
    c = np.loadtxt(folder+'GA/centroids '+sub+'.out')
    h = np.loadtxt(folder+'GA/histogramREF '+sub+'.out')
    h1  = []
    modified = False
    if not modified :
        h1 = h
        print("do not change")
    else:
        print("change")
    npH1 = np.array(h1)
    hGA.append(h1)
    ma = np.amax(npH1, axis=1).tolist()
    me = np.median(np.around(npH1, decimals=2), axis=1).tolist()
    avg = np.average(npH1, axis=1).tolist()
    mp = []
    for his in h1:
        mp.append(np.array(np.where( np.array(abs(his-his[0])) <2 )).shape[1]*100/len(his))
    
    print(mp)
    GA_motionfreeFile.append(mp)    
    GA_centrFile.append(c)
    GA_maxFile.append(ma)
    GA_avgFile.append(avg)
    GA_medianFile.append(me)  
    GA_patients.append(sub)
    

Look at all motion data to make sure that imported correctly:

In [ ]:
for data, pat in zip(hGA, GA_patients):
    plt.plot(data.T)
    plt.title(pat)
    plt.xlabel('Frame Number (not continuous)')
    plt.ylabel('Displacement from reference')
    plt.show()

In [ ]:
# split up into variable for 16 regions and variable for PointCloud centroid:
GA_max_PCl = []
GA_avg_PCl = []
GA_centr_PCl = []
GA_median_PCl = []
GA_motionfree_PCl = []
for i in range(len(GA_maxFile)):
    GA_max_PCl.append(GA_maxFile[i][-1])
    GA_maxFile[i] = GA_maxFile[i][:-1]
    GA_avg_PCl.append(GA_avgFile[i][-1])
    GA_avgFile[i] = GA_avgFile[i][:-1]
    GA_centr_PCl.append(GA_centrFile[i][-1])
    GA_centrFile[i] = GA_centrFile[i][:-1]
    GA_median_PCl.append(GA_medianFile[i][-1])
    GA_medianFile[i] = GA_medianFile[i][:-1]
    GA_motionfree_PCl.append(GA_motionfreeFile[i][-1])
    GA_motionfreeFile[i] = GA_motionfreeFile[i][:-1]

- hGA is list of displacement for 16 regions of 18 GA subjects 
- GA_maxFile, GA_avgFile, GA_medianFile contain maximum, mean and median value of displacement for each subject and each region
- GA_max_PCl, ... contain maximum, mean and median value of displacemnt for each subject's point cloud centroid
- GA_patients contains list of patients acronym



Same analysis for no GA subjects:

In [ ]:
'''#save to file no GA info / point cloud centroid added to centroids 
for sub in no_GA:
    print(sub)
    # reads the vox2ras for the corresponding subject
    vox2ras = np.array(np.loadtxt(folder+"v2r/"+sub+".txt", dtype=float)).reshape(4,4)
    c_,r,_ = getCentriods(folder+"regions binarization/"+sub,vox2ras)
    pcl_centr = PClCentroid(sub)
    print('PointCloud centroid: ', pcl_centr)
    c = np.append(c_, [pcl_centr], axis=0)
    np.savetxt(folder+'No GA/centroids '+sub+'.out', np.array(c))
    mat_files = StartEndFrameNr([sub], Scan_Times, './new_calc/No GA/')
    h = computeHistograms2(mat_files,c)
    np.savetxt(folder+'No GA/histogramREF '+sub+'.out', np.array(h))'''

In [ ]:
noGAcentrFile = []
noGA_maxFile = []
noGA_avgFile = []
noGA_medianFile = []
noGA_motionfreeFile = []
noGA_patients = []
h_noGA = []
for sub in no_GA:
    print(sub)
    c = np.loadtxt(folder+'No GA/centroids '+sub+'.out')
    h = np.loadtxt(folder+'No GA/histogramREF '+sub+'.out')
    h1 = []
    npH = np.array(h)
    exclude = False
    count = 0 
            
            
    if count!=16:
        h1 = h
        print(count)
        print("dont change")
    else:
        print("change")
    h_noGA.append(h1)
    npH1 = np.array(h1)     
 
    ma = np.amax(npH1, axis=1).tolist()
    me = np.median(np.around(npH1, decimals=2), axis=1).tolist()
    print(me)
    avg = np.average(npH1, axis=1).tolist()
    mp = []
    for his in h1:
        mp.append(np.array(np.where( np.array(abs(his-his[0])) <2 )).shape[1]*100/len(his)) 
        
    print(mp)
    noGA_motionfreeFile.append(mp) 
    noGAcentrFile.append(c)
    noGA_maxFile.append(ma)
    noGA_avgFile.append(avg)
    noGA_medianFile.append(me)
    noGA_patients.append(sub)

In [ ]:
for data, pat in zip(h_noGA, noGA_patients):
    plt.plot(data.T)
    plt.title(pat)
    plt.xlabel('Frame Number (not continuous)')
    plt.ylabel('Displacement from reference')
    plt.show()

In [ ]:
# split up into variable for 16 regions and variable for PointCloud centroid:
noGA_max_PCl = []
noGA_avg_PCl = []
noGA_centr_PCl = []
noGA_median_PCl = []
noGA_motionfree_PCl = []
for i in range(len(noGA_maxFile)):
    noGA_max_PCl.append(noGA_maxFile[i][-1])
    noGA_maxFile[i] = noGA_maxFile[i][:-1]
    noGA_avg_PCl.append(noGA_avgFile[i][-1])
    noGA_avgFile[i] = noGA_avgFile[i][:-1]
    noGA_centr_PCl.append(noGAcentrFile[i][-1])
    noGAcentrFile[i] = noGAcentrFile[i][:-1]
    noGA_median_PCl.append(noGA_medianFile[i][-1])
    noGA_medianFile[i] = noGA_medianFile[i][:-1]
    noGA_motionfree_PCl.append(noGA_motionfreeFile[i][-1])
    noGA_motionfreeFile[i] = noGA_motionfreeFile[i][:-1]

In [ ]:
print(np.amax(GA_avg_PCl), np.amax(GA_median_PCl), np.amax(GA_max_PCl), np.amin(GA_motionfree_PCl))
print(np.median(GA_avg_PCl), np.median(GA_median_PCl), np.median(GA_max_PCl), np.median(GA_motionfree_PCl))

## 2. Compare metrics for point cloud centroid:

In [ ]:
metrics_GA = [GA_avg_PCl, GA_median_PCl, GA_max_PCl, GA_motionfree_PCl]
metrics_noGA = [noGA_avg_PCl, noGA_median_PCl, noGA_max_PCl, noGA_motionfree_PCl]
titles = ['Mean Displacement', 'Median Displacement', 'Maximum Displacement', 'Motion-free Time']
ylabels = ['Displacement [mm]', 'Displacement [mm]', 'Displacement [mm]', 'Motion-free time [%]']
colors = ['tab:blue', 'tab:orange']

plt.figure(figsize=(10,7))
plt.rcParams.update({'font.size': 14})
for i in range(0, 4):
    plt.subplot(2,2,i+1)
    plt.boxplot([metrics_noGA[i], metrics_GA[i]])
    for k, j in zip(range(1, 3), [metrics_noGA[i], metrics_GA[i]]):
        plt.plot(np.ones(len(j))*k, j, 'o', c=colors[k-1], ms=4)
    plt.title(titles[i])
    plt.xticks([1,2], ['No GA', 'GA'])
    plt.ylabel(ylabels[i])
plt.subplots_adjust(hspace=0.4, wspace=0.35)
plt.savefig(folder+'Plots/Metrics_PClCentroid.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
print(np.median(noGA_avg_PCl), np.std(noGA_avg_PCl))
print(np.median(GA_avg_PCl), np.std(GA_avg_PCl))

In [ ]:
print('No GA:')
print('Med:', np.median(noGA_median_PCl), '+-', np.std(noGA_median_PCl))
print('Mean:', np.median(noGA_avg_PCl), '+-', np.std(noGA_avg_PCl))
print('Max:', np.median(noGA_max_PCl), '+-', np.std(noGA_max_PCl))
print('Motionfree:', np.median(noGA_motionfree_PCl), '+-', np.std(noGA_motionfree_PCl))
print('GA:')
print('Med:', np.median(GA_median_PCl), '+-', np.std(GA_median_PCl))
print('Mean:', np.median(GA_avg_PCl), '+-', np.std(GA_avg_PCl))
print('Max:', np.median(GA_max_PCl), '+-', np.std(GA_max_PCl))
print('Motionfree:', np.median(GA_motionfree_PCl), '+-', np.std(GA_motionfree_PCl))

Plot comparison of excluded and included.

First run analysis for excluded scans at the end of this notebook!

In [ ]:
metrics_GA_ex = np.loadtxt('../new_calc/metrics_GA_Excl')
metrics_noGA_ex = np.loadtxt('../new_calc/metrics_noGA_Excl')

titles = ['Mean Displacement', 'Median Displacement', 'Maximum Displacement', 'Motion-free Time']
ylabels = ['Displacement [mm]', 'Displacement [mm]', 'Displacement [mm]', 'Motion-free time [%]']
colors = ['tab:blue', 'tab:gray', 'tab:orange', 'tab:gray']

plt.figure(figsize=(11,7))
plt.rcParams.update({'font.size': 14})
for i in range(0, 4):
    plt.subplot(2,2,i+1)
    plt.boxplot([metrics_noGA[i], metrics_noGA_ex[i], metrics_GA[i], metrics_GA_ex[i]])
    for k, j in zip(range(1, 5), [metrics_noGA[i], metrics_noGA_ex[i], metrics_GA[i], metrics_GA_ex[i]]):
        plt.plot(np.ones(len(j))*k, j, 'o', c=colors[k-1], ms=4)
    plt.title(titles[i])
    plt.xticks([1,2,3,4], ['No GA', 'No GA Excl', 'GA', 'GA Excl'], fontsize=12)
    plt.ylabel(ylabels[i])
plt.subplots_adjust(hspace=0.4, wspace=0.35)
plt.savefig(folder+'Plots/App_Excl_Metrics_PClCentroid.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
age_, pat_id_ = np.loadtxt(folder+'ages.csv', skiprows=1, dtype=str, unpack=True, delimiter=',')

pat_id, age = [], []
for i in range(len(GA_patients)):
    ind = np.where(pat_id_ == GA_patients[i])[0][0]
    pat_id.append(pat_id_[ind])
    age.append(int(age_[ind]))

In [ ]:
# distinguish between GA under/over 7 years
ind1 = np.where(np.array(age) <= 7)
ind2 = np.where(np.array(age) > 7)
metrics_GA_und = [np.array(GA_avg_PCl)[ind1], np.array(GA_median_PCl)[ind1], np.array(GA_max_PCl)[ind1], np.array(GA_motionfree_PCl)[ind1]]
metrics_GA_ov = [np.array(GA_avg_PCl)[ind2], np.array(GA_median_PCl)[ind2], np.array(GA_max_PCl)[ind2], np.array(GA_motionfree_PCl)[ind2]]
metrics_noGA = [noGA_avg_PCl, noGA_median_PCl, noGA_max_PCl, noGA_motionfree_PCl]
titles = ['Mean Displacement', 'Median Displacement', 'Maximum Displacement', 'Motion-free Time']
ylabels = ['Displacement [mm]', 'Displacement [mm]', 'Displacement [mm]', 'Motion-free time [%]']
colors = ['tab:blue', 'tab:orange', 'tab:green']

plt.figure(figsize=(10,7))
plt.rcParams.update({'font.size': 14})
for i in range(0, 4):
    plt.subplot(2,2,i+1)
    plt.boxplot([metrics_noGA[i], metrics_GA_und[i], metrics_GA_ov[i]])
    for k, j in zip(range(1, 4), [metrics_noGA[i], metrics_GA[i], metrics_GA_ov[i]]):
        plt.plot(np.ones(len(j))*k, j, 'o', c=colors[k-1], ms=4)
    plt.title(titles[i])
    plt.xticks([1,2, 3], ['No GA', 'GA $\leq$ 7y', 'GA > 7y'])
    plt.ylabel(ylabels[i])
plt.subplots_adjust(hspace=0.4, wspace=0.35)
plt.show()

### Statistics:
only distinguish between no GA and GA, but if age is taken into account in paper as well, needs to be included!

In [ ]:
p_values = []
tests = []
metrics_GA = [GA_avg_PCl, GA_median_PCl, GA_max_PCl, GA_motionfree_PCl]
metrics_noGA = [noGA_avg_PCl, noGA_median_PCl, noGA_max_PCl, noGA_motionfree_PCl]
name = ['PCl_avg', 'PCl_median', 'PCl_max', 'PCl_motionfree']
altern = ['less', 'less', 'less', 'greater']
print('Test the four matrices:')
for mga, mnga, n, alt in zip(metrics_GA, metrics_noGA, name, altern):
    p = mannwhitneyu(mga, mnga, alternative=alt)[1]
    p_values.append(p)
    tests.append(n)
    print(n, ' test GA ', alt, ' no GA: p = ', p)



## 4. Movement on axes:

In [ ]:
'''# extract motion separately on each axis:
allx = []
ally = []
allz = []
for sub in GA:
    xx = []
    yy = []
    zz = []
    centr = np.loadtxt(folder+'GA/centroids '+sub+'.out')
    filenames = StartEndFrameNr([sub], Scan_Times, folder+'GA/')
    for centroid in centr:
        x = []
        y = []
        z = []
        for file in filenames:
            mat = scipy.io.loadmat(file)
            if "ref" not in file:
                transf = mat["matrix"]
                ec = np.append(centroid,1)
                newCentroid = np.dot(transf, ec)
                x.append((ec[0]-newCentroid[0]))
                y.append((ec[1]-newCentroid[1]))
                z.append((ec[2]-newCentroid[2]))
        xx.append(x)
        yy.append(y)
        zz.append(z)
        print(sub)
    allx.append(xx)
    ally.append(yy)
    allz.append(zz)
    
count = 0
for sub in GA:
    np.savetxt(folder+'GA axis/xREF '+sub+'.out', np.array(allx[count]))
    np.savetxt(folder+'GA axis/yREF '+sub+'.out', np.array(ally[count]))
    np.savetxt(folder+'GA axis/zREF '+sub+'.out', np.array(allz[count]))
    count+=1'''

In [ ]:
'''# extract motion separately on each axis:
allx1 = []
ally1 = []
allz1 = []
for sub in no_GA:
    xx = []
    yy = []
    zz = []
    centr = np.loadtxt(folder+'No GA/centroids '+sub+'.out')
    filenames = StartEndFrameNr([sub], Scan_Times, folder+'No GA/')
    for centroid in centr:
        x = []
        y = []
        z = []
        for file in filenames:
            mat = scipy.io.loadmat(file)
            if "ref" not in file:
                transf = mat["matrix"]
                ec = np.append(centroid,1)
                newCentroid = np.dot(transf, ec)
                x.append((ec[0]-newCentroid[0]))
                y.append((ec[1]-newCentroid[1]))
                z.append((ec[2]-newCentroid[2]))
        xx.append(x)
        yy.append(y)
        zz.append(z)
        print(sub)
    allx1.append(xx)
    ally1.append(yy)
    allz1.append(zz)
 
count = 0
for sub in no_GA:
    np.savetxt(folder+'No GA axis/xREF '+sub+'.out', np.array(allx1[count]))
    np.savetxt(folder+'No GA axis/yREF '+sub+'.out', np.array(ally1[count]))
    np.savetxt(folder+'No GA axis/zREF '+sub+'.out', np.array(allz1[count]))
    count+=1'''

In [ ]:
# load the data and calculate medians:
noGA_medianx = []
noGA_mediany = []
noGA_medianz = []
allx1 = []
ally1 = []
allz1 = []
for sub in no_GA:
    xx = []
    yy = []
    zz = []
    print(sub)
    x = abs(np.loadtxt(folder+'No GA axis/xREF '+sub+'.out'))
    y = abs(np.loadtxt(folder+'No GA axis/yREF '+sub+'.out'))
    z = abs(np.loadtxt(folder+'No GA axis/zREF '+sub+'.out'))
    change = False            
            
    if change == True:
        x = xx
        y = yy
        z = zz
        print("change")
    allx1.append(x)
    ally1.append(y)
    allz1.append(z)
    mex = np.median(np.around(np.array(x), decimals=2), axis=1).tolist()
    mey = np.median(np.around(np.array(y), decimals=2), axis=1).tolist()
    mez = np.median(np.around(np.array(z), decimals=2), axis=1).tolist()

    noGA_medianx.append(mex)
    noGA_mediany.append(mey)
    noGA_medianz.append(mez)

In [ ]:
# load the data and calculate medians:
GA_medianx = []
GA_mediany = []
GA_medianz = []
allx = []
ally = []
allz = []
a = 0
for sub in GA:
    xx = []
    yy = []
    zz = []
    print(sub)
    x = abs(np.loadtxt(folder+'GA axis/xREF '+sub+'.out'))
    y = abs(np.loadtxt(folder+'GA axis/yREF '+sub+'.out'))
    z = abs(np.loadtxt(folder+'GA axis/zREF '+sub+'.out'))

    change = False

    if change == True:
        x = xx
        y = yy
        z = zz
        print("change")
    allx.append(x)
    ally.append(y)
    allz.append(z)
    mex = np.median(np.around(np.array(x), decimals=2), axis=1).tolist()
    mey = np.median(np.around(np.array(y), decimals=2), axis=1).tolist()
    mez = np.median(np.around(np.array(z), decimals=2), axis=1).tolist()

    GA_medianx.append(mex)
    GA_mediany.append(mey)
    GA_medianz.append(mez)
    a+=1

## 5. Decomposition of matrices:

In [ ]:
GA_MeanTr_pm, GA_MeanRot_pm, GA_StdTr_pm, GA_StdRot_pm = Decompose_All_pm(GA_patients, path=folder+'GA/')        
print('first done')
noGA_MeanTr_pm, noGA_MeanRot_pm, noGA_StdTr_pm, noGA_StdRot_pm = Decompose_All_pm(noGA_patients, path=folder+'No GA/')
print('both done')

In [ ]:
GA_MeanTr, GA_MeanRot, GA_StdTr, GA_StdRot = Decompose_All(GA_patients, path=folder+'GA/')        
print('first done')
noGA_MeanTr, noGA_MeanRot, noGA_StdTr, noGA_StdRot = Decompose_All(noGA_patients, path=folder+'No GA/')
print('both done')

In [ ]:
# Test translations / rotations for GA group:
print('Test with GA group:')
p = mannwhitneyu(GA_MeanTr[:,2], GA_MeanTr[:,1],alternative='greater')[1]
p_values.append(p)
tests.append('GA_T_z_greater_y')
print('Tz with Ty', p)
p = mannwhitneyu(GA_MeanTr[:,2], GA_MeanTr[:,0],alternative='greater')[1]
p_values.append(p)
tests.append('GA_T_z_greater_x')
print('Tz with Tx', p)
p = mannwhitneyu(GA_MeanTr[:,0], GA_MeanTr[:,1],alternative='greater')[1]
p_values.append(p)
tests.append('GA_T_x_greater_y')
print('Tx with Ty', p)
p = mannwhitneyu(GA_MeanRot[:,2], GA_MeanRot[:,1],alternative='greater')[1]
p_values.append(p)
tests.append('GA_R_z_greater_y')
print('Rz with Ry', p)
p = mannwhitneyu(GA_MeanRot[:,0], GA_MeanRot[:,2],alternative='greater')[1]
p_values.append(p)
tests.append('GA_R_x_greater_z')
print('Rx with Rz', p)
p = mannwhitneyu(GA_MeanRot[:,0], GA_MeanRot[:,1],alternative='greater')[1]
p_values.append(p)
tests.append('GA_R_x_greater_y')
print('Rx with Ry', p)

# Test translations / rotations for no GA group:
print('Test without GA group:')
p = mannwhitneyu(noGA_MeanTr[:,2], noGA_MeanTr[:,1],alternative='greater')[1]
p_values.append(p)
tests.append('noGA_T_z_greater_y')
print('Tz with Ty', p)
p = mannwhitneyu(noGA_MeanTr[:,2], noGA_MeanTr[:,0],alternative='greater')[1]
p_values.append(p)
tests.append('noGA_T_z_greater_x')
print('Tz with Tx', p)
p = mannwhitneyu(noGA_MeanTr[:,0], noGA_MeanTr[:,1],alternative='greater')[1]
p_values.append(p)
tests.append('noGA_T_x_greater_y')
print('Tx with Ty', p)
p = mannwhitneyu(noGA_MeanRot[:,2], noGA_MeanRot[:,1],alternative='greater')[1]
p_values.append(p)
tests.append('noGA_R_z_greater_y')
print('Rz with Ry', p)
p = mannwhitneyu(noGA_MeanRot[:,0], noGA_MeanRot[:,2],alternative='greater')[1]
p_values.append(p)
tests.append('noGA_R_x_greater_z')
print('Rx with Rz', p)
p = mannwhitneyu(noGA_MeanRot[:,0], noGA_MeanRot[:,1],alternative='greater')[1]
p_values.append(p)
tests.append('noGA_R_x_greater_y')
print('Rx with Ry', p)

In [ ]:
print(np.median(noGA_MeanTr_pm, axis=0))
print(np.std(noGA_MeanTr_pm, axis=0))
print(np.median(GA_MeanTr_pm, axis=0))
print(np.std(GA_MeanTr_pm, axis=0))
print(np.median(noGA_MeanRot_pm, axis=0))
print(np.std(noGA_MeanRot_pm, axis=0))

In [ ]:
print(np.median(noGA_MeanTr, axis=0))
print(np.std(noGA_MeanTr, axis=0))
print(np.median(GA_MeanTr, axis=0))
print(np.std(GA_MeanTr, axis=0))
print(np.median(noGA_MeanRot, axis=0))
print(np.std(noGA_MeanRot, axis=0))

# 6. Multiple Tests correction:

In [ ]:
# add p-values for motion on axes:
ax = []
centroidsMel, regionsMel,result = getCentriods(folder+"test_data",vox2ras,0)
for i in range(16):
    print(regionsMel[i])
    #print(mannwhitneyu(np.array(GA_medianz)[:,i],np.array(GA_medianx)[:,i],alternative='greater')[1]<(0.05/64))    
    p_values.append(mannwhitneyu(np.array(GA_medianz)[:,i],np.array(GA_medianx)[:,i],alternative='greater')[1])
    tests.append(regionsMel[i]+'GA_zx')
    #print(mannwhitneyu(np.array(GA_medianz)[:,i],np.array(GA_mediany)[:,i],alternative='greater')[1]<(0.05/64))
    p_values.append(mannwhitneyu(np.array(GA_medianz)[:,i],np.array(GA_mediany)[:,i],alternative='greater')[1])
    tests.append(regionsMel[i]+'GA_zy')
    #print(mannwhitneyu(np.array(noGA_medianz)[:,i],np.array(noGA_medianx)[:,i],alternative='greater')[1]<(0.05/64))    
    p_values.append(mannwhitneyu(np.array(noGA_medianz)[:,i],np.array(noGA_medianx)[:,i],alternative='greater')[1])
    tests.append(regionsMel[i]+'noGA_zx')
    #print(mannwhitneyu(np.array(noGA_medianz)[:,i],np.array(noGA_mediany)[:,i],alternative='greater')[1]<(0.05/64))
    p_values.append(mannwhitneyu(np.array(noGA_medianz)[:,i],np.array(noGA_mediany)[:,i],alternative='greater')[1])
    tests.append(regionsMel[i]+'noGA_zy')

In [ ]:
#add p_values for metrics:
for i in range(16):
    print(regionsMel[i],0.05/54)
    #print(mannwhitneyu(np.array(noGA_maxFile)[:,i],np.array(GA_maxFile)[:,i],alternative='greater')[1])
    p_values.append(mannwhitneyu(np.array(noGA_maxFile)[:,i],np.array(GA_maxFile)[:,i],alternative='greater')[1])
    tests.append(regionsMel[i]+'max')
    #print(mannwhitneyu(np.array(noGA_medianFile)[:,i],np.array(GA_medianFile)[:,i],alternative='greater')[1])
    p_values.append(mannwhitneyu(np.array(noGA_medianFile)[:,i],np.array(GA_medianFile)[:,i],alternative='greater')[1])
    tests.append(regionsMel[i]+'median')
    #print(mannwhitneyu(np.array(noGA_avgFile)[:,i],np.array(GA_avgFile)[:,i],alternative='greater')[1])
    p_values.append(mannwhitneyu(np.array(noGA_avgFile)[:,i],np.array(GA_avgFile)[:,i],alternative='greater')[1])
    tests.append(regionsMel[i]+'avg')
    #print(mannwhitneyu(np.array(noGA_motionfreeFile)[:,i],np.array(GA_motionfreeFile)[:,i],alternative='less')[1])
    p_values.append(mannwhitneyu(np.array(noGA_motionfreeFile)[:,i],np.array(GA_motionfreeFile)[:,i],alternative='less')[1])
    tests.append(regionsMel[i]+'motionfree')

In [ ]:
rej, p_values_cor, _, __ = multipletests(p_values, alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)

In [ ]:
for i in range(len(p_values)):
    print(' reject = ', rej[i], ' ', tests[i], ' p = ', p_values[i], ' p_cor = ', p_values_cor[i])

### Comparison metrics GA and non GA:

In [ ]:
metrics_GA = [GA_avg_PCl, GA_median_PCl, GA_max_PCl, GA_motionfree_PCl]
metrics_noGA = [noGA_avg_PCl, noGA_median_PCl, noGA_max_PCl, noGA_motionfree_PCl]
titles = ['Mean Displacement', 'Median Displacement', 'Maximum Displacement', 'Motion-free Time']
ylabels = ['Displacement [mm]', 'Displacement [mm]', 'Displacement [mm]', 'Motion-free time [%]']
colors = ['tab:blue', 'tab:orange']
p_cor = p_values_cor[0:4]
star = p_cor < 0.05
starstar = p_cor < 0.001
print(p_cor, star, starstar)


plt.figure(figsize=(10,7))
plt.rcParams.update({'font.size': 14})
for i in range(0, 4):
    ax = plt.subplot(2,2,i+1)
    plt.boxplot([metrics_noGA[i], metrics_GA[i]])
    for k, j in zip(range(1, 3), [metrics_noGA[i], metrics_GA[i]]):
        plt.plot(np.ones(len(j))*k, j, 'o', c=colors[k-1], ms=4)
    plt.title(titles[i])
    plt.xticks([1,2], ['No GA', 'GA'])
    plt.ylabel(ylabels[i])
    bars = np.array([1,2])
    heights = np.array([np.amax(metrics_noGA[i]), np.amax(metrics_GA[i])])
    if starstar[i]==True:
        barplot_annotate_brackets(0, 1, '**', bars, heights, dh=.1)
    else:
        if star[i]==True:
            barplot_annotate_brackets(0, 1, '*', bars, heights, dh=.1)
    lim = ax.get_ylim()
    plt.ylim(lim[0],(lim[1]-lim[0])*1.15)
plt.subplots_adjust(hspace=0.4, wspace=0.35)
plt.savefig(folder+'Plots/Metrics_PClCentroid.png', dpi=200, bbox_inches='tight')
plt.show()

### Transformation decomposition:

In [ ]:
p_cor = p_values_cor[4:16]
star = p_cor < 0.05
starstar = p_cor < 0.001
print(p_cor, star, starstar)
a = [1,0,0,4,3,3]
b = [2,2,1,5,5,4]

colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:gray', 'tab:cyan', 
          'tab:olive', 'firebrick', 'gold', 'mediumaquamarine', 'purple']

plt.figure(figsize=(13, 5))
plt.rcParams.update({'font.size': 14})
ax = plt.subplot(1,2,1)
plt.boxplot(np.append(noGA_MeanTr_pm, noGA_MeanRot_pm, axis=1))
for i, j, k in zip(range(1, 7), np.append(noGA_MeanTr_pm, noGA_MeanRot_pm, axis=1).T, ['tx', 'ty', 'tz', 'Rx', 'Ry', 'Rz']):
    plt.plot(np.ones(len(j))*i, j, 'o', c=colors[i-1], label=k, ms=4)
plt.ylabel('Mean translation / rotation [mm / °]')
plt.xticks([1,2,3,4,5,6], ['$t_x$', '$t_y$', '$t_z$', '$R_x$', '$R_y$', '$R_z$'])
plt.title('Patients without GA')
bars = np.array([1,2,3,4,5,6])
heights = np.append(np.amax(noGA_MeanTr_pm, axis=0), np.amax(noGA_MeanRot_pm, axis=0))
cnt1, cnt2 = 0,0
dh=[.3, .2, .4]
for i,a_,b_ in zip(range(6,12),a,b):
    if starstar[i]==True:
        barplot_annotate_brackets(a_, b_, '**', bars, heights, dh=dh[cnt1])
        cnt1+=1
    else:
        if star[i]==True:
            barplot_annotate_brackets(a_, b_, '*', bars, heights, dh=dh[cnt2])
            cnt2+=1
lim = ax.get_ylim()
plt.ylim(lim[0],(lim[1])*1.05)
            
ax=plt.subplot(1,2,2)
plt.boxplot(np.append(GA_MeanTr_pm, GA_MeanRot_pm, axis=1))
for i, j, k in zip(range(1, 7), np.append(GA_MeanTr_pm, GA_MeanRot_pm, axis=1).T, ['tx', 'ty', 'tz', 'Rx', 'Ry', 'Rz']):
    plt.plot(np.ones(len(j))*i, j, 'o', c=colors[i-1], label=k, ms=4)
plt.ylabel('Mean translation / rotation [mm / °]')
plt.xticks([1,2,3,4,5,6], ['$t_x$', '$t_y$', '$t_z$', '$R_x$', '$R_y$', '$R_z$'])
plt.title('Patients with GA')
bars = np.array([1,2,3,4,5,6])
heights = np.append(np.amax(GA_MeanTr_pm, axis=0), np.amax(GA_MeanRot_pm, axis=0))
cnt1, cnt2 = 0,0
dh=[.3, .2, .3]
for i,a_,b_ in zip(range(0,6),a,b):
    if starstar[i]==True:
        barplot_annotate_brackets(a_, b_, '**', bars, heights, dh=dh[cnt1])
        cnt1+=1
    else:
        if star[i]==True:
            barplot_annotate_brackets(a_, b_, '*', bars, heights, dh=dh[cnt2])
            cnt2+=1
lim = ax.get_ylim()
plt.ylim(lim[0],(lim[1])*1.05)
plt.subplots_adjust(wspace=0.25)
plt.savefig(folder+'Plots/Transf_Decomposition.png', dpi=200, bbox_inches='tight')
plt.show()

### Motion on axes:

In [ ]:
p_cor = p_values_cor[16:80]
star = p_cor < 0.05
starstar = p_cor < 0.001

maxlim = 35
medlim = 5
avglim = 5

In [ ]:
data1s = [noGA_medianx, GA_medianx]
data2s = [noGA_mediany, GA_mediany]
data3s = [noGA_medianz, GA_medianz]
GAInfo = ['without GA', 'with GA']
bars = np.arange(1, 13, 1)
heights_nGA_ = np.amax(noGA_medianz, axis=0)
heights_nGA=[]
for h in heights_nGA_:
    heights_nGA.append(h)
    heights_nGA.append(h)
    heights_nGA.append(h)
heights_GA_ = np.amax(GA_medianz, axis=0)
heights_GA=[]
for h in heights_GA_:
    heights_GA.append(h)
    heights_GA.append(h)
    heights_GA.append(h)
heights = [heights_nGA, heights_GA]
const = [2,0]

plt.figure(figsize=(15,18))
nr=1
for data1, data2, data3, ga, h, c in zip(data1s, data2s, data3s, GAInfo, heights, const):
    title = "Patients "+ga+", subcortical regions"
    MakeSubPlot(np.array(data1), np.array(data2), np.array(data3),"x","y","z",title,medlim+1.5,0,4,nr,1)
    for i,a in zip(np.array([0,4,8,12])+c, [0,3,6,9]):
        if starstar[i]==True:
            barplot_annotate_brackets(a, a+2, '**', bars, h, dh=0.2)
        else:
            if star[i]==True:
                barplot_annotate_brackets(a, a+2, '*', bars, h, dh=0.2)
        if starstar[i+1]==True:
            barplot_annotate_brackets(a+1, a+2, '**', bars, h, dh=0.1)
        else:
            if star[i+1]==True:
                barplot_annotate_brackets(a+1, a+2, '*', bars, h, dh=0.1)
    nr+=1
    title = "Patients "+ga+", cortical regions"
    MakeSubPlot(np.array(data1), np.array(data2), np.array(data3),"x","y","z",title,medlim+1.5,8,12,nr)
    for i,a in zip(np.array([32,36,40,44])+c, [0,3,6,9]):
        if starstar[i]==True:
            barplot_annotate_brackets(a, a+2, '**', bars, h[24:], dh=0.2)
        else:
            if star[i]==True:
                barplot_annotate_brackets(a, a+2, '*', bars, h[24:], dh=0.2)
        if starstar[i+1]==True:
            barplot_annotate_brackets(a+1, a+2, '**', bars, h[24:], dh=0.1)
        else:
            if star[i+1]==True:
                barplot_annotate_brackets(a+1, a+2, '*', bars, h[24:], dh=0.1)
    
    nr+=1
plt.subplots_adjust(hspace=0.4, wspace=0.35)
plt.savefig(folder+'Plots/MotionOnAxes.png', dpi=200, bbox_inches='tight')
plt.savefig(folder+'Plots/MotionOnAxes.eps', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
data1s = [noGA_medianx, GA_medianx]
data2s = [noGA_mediany, GA_mediany]
data3s = [noGA_medianz, GA_medianz]
GAInfo = ['without GA', 'with GA']
bars = np.arange(1, 13, 1)
heights_nGA_ = np.amax(noGA_medianz, axis=0)
heights_nGA=[]
for h in heights_nGA_:
    heights_nGA.append(h)
    heights_nGA.append(h)
    heights_nGA.append(h)
heights_GA_ = np.amax(GA_medianz, axis=0)
heights_GA=[]
for h in heights_GA_:
    heights_GA.append(h)
    heights_GA.append(h)
    heights_GA.append(h)
heights = [heights_nGA, heights_GA]
const = [2,0]

plt.figure(figsize=(15,18))
nr=1
for data1, data2, data3, ga, h, c in zip(data1s, data2s, data3s, GAInfo, heights, const):
    title = "Patients "+ga+", subcortical regions"
    MakeSubPlot(np.array(data1), np.array(data2), np.array(data3),"x","y","z",title,medlim+1.5,4,8,nr,1)
    for i,a in zip(np.array([16,20,24,28])+c, [0,3,6,9]):
        if starstar[i]==True:
            barplot_annotate_brackets(a, a+2, '**', bars, h[12:], dh=0.2)
        else:
            if star[i]==True:
                barplot_annotate_brackets(a, a+2, '*', bars, h[12:], dh=0.2)
        if starstar[i+1]==True:
            barplot_annotate_brackets(a+1, a+2, '**', bars, h[12:], dh=0.1)
        else:
            if star[i+1]==True:
                barplot_annotate_brackets(a+1, a+2, '*', bars, h[12:], dh=0.1)
    nr+=1
    title = "Patients "+ga+", cortical regions"
    MakeSubPlot(np.array(data1), np.array(data2), np.array(data3),"x","y","z",title,medlim+1.5,12,16,nr)
    for i,a in zip(np.array([48,52,56,60])+c, [0,3,6,9]):
        if starstar[i]==True:
            barplot_annotate_brackets(a, a+2, '**', bars, h[36:], dh=0.2)
        else:
            if star[i]==True:
                barplot_annotate_brackets(a, a+2, '*', bars, h[36:], dh=0.2)
        if starstar[i+1]==True:
            barplot_annotate_brackets(a+1, a+2, '**', bars, h[36:], dh=0.1)
        else:
            if star[i+1]==True:
                barplot_annotate_brackets(a+1, a+2, '*', bars, h[36:], dh=0.1)
    nr+=1
plt.subplots_adjust(hspace=0.4, wspace=0.35)
plt.savefig(folder+'Plots/App_MotionOnAxes.png', dpi=200, bbox_inches='tight')
plt.savefig(folder+'Plots/App_MotionOnAxes.eps', dpi=200, bbox_inches='tight')
plt.show()

### Metrics for each region:

In [ ]:
p_cor = p_values_cor[80:]
print(tests[80:])
star = p_cor < 0.05
starstar = p_cor < 0.001
print(p_cor, star, starstar)

In [ ]:
t = tests[80:]
for i, a in zip(np.arange(32, 64, 4), np.arange(0,17,2)):
    print(t[i],a)

In [ ]:
mflim = 120
maxlim = 22

bars = np.arange(1, 17, 1)

# max
heights_ = np.amax(noGA_maxFile, axis=0)
heights=[]
for h in heights_:
    heights.append(h)
    heights.append(h)
plt.figure(figsize=(15,9))
plt.subplot(2,1,1)
makePlotCh(np.array(GA_maxFile)[:,:8],np.array(noGA_maxFile)[:,:8],"GA","No GA","Subcortical regions",maxlim,1)
for i, a in zip(np.arange(0, 29, 4), np.arange(0,17,2)):
    if starstar[i]==True:
        barplot_annotate_brackets(a, a+1, '**', bars, heights, dh=0.1)
    else:
        if star[i]==True:
            barplot_annotate_brackets(a, a+1, '*', bars, heights, dh=0.1)
plt.subplot(2,1,2)
makePlotCh(np.array(GA_maxFile)[:,8:],np.array(noGA_maxFile)[:,8:],"GA","No GA","Cortical regions",maxlim, leg=True)
for i, a in zip(np.arange(32, 64, 4), np.arange(0,17,2)):
    if starstar[i]==True:
        barplot_annotate_brackets(a, a+1, '**', bars, heights[16:], dh=0.1)
    else:
        if star[i]==True:
            barplot_annotate_brackets(a, a+1, '*', bars, heights[16:], dh=0.1)
plt.subplots_adjust(hspace=0.4, wspace=0.35)
plt.savefig(folder+'Plots/App_MaxRegions.png', dpi=200, bbox_inches='tight')
plt.show()

# median
heights_ = np.amax(noGA_medianFile, axis=0)
heights=[]
for h in heights_:
    heights.append(h)
    heights.append(h)
plt.figure(figsize=(15,9))
plt.subplot(2,1,1)
makePlotCh(np.array(GA_medianFile)[:,:8],np.array(noGA_medianFile)[:,:8],"GA","No GA","Subcortical regions",medlim+1.7,1)
for i, a in zip(np.arange(1, 30, 4), np.arange(0,17,2)):
    if starstar[i]==True:
        barplot_annotate_brackets(a, a+1, '**', bars, heights, dh=0.1)
    else:
        if star[i]==True:
            barplot_annotate_brackets(a, a+1, '*', bars, heights, dh=0.1)
plt.subplot(2,1,2)
makePlotCh(np.array(GA_medianFile)[:,8:],np.array(noGA_medianFile)[:,8:],"GA","No GA","Cortical regions",medlim+1.7,leg=True)
for i, a in zip(np.arange(33, 65, 4), np.arange(0,17,2)):
    if starstar[i]==True:
        barplot_annotate_brackets(a, a+1, '**', bars, heights[16:], dh=0.1)
    else:
        if star[i]==True:
            barplot_annotate_brackets(a, a+1, '*', bars, heights[16:], dh=0.1)
plt.subplots_adjust(hspace=0.4, wspace=0.35)
plt.savefig(folder+'Plots/App_MedianRegions.png', dpi=200, bbox_inches='tight')
plt.show()

# average
heights_ = np.amax(noGA_avgFile, axis=0)
heights=[]
for h in heights_:
    heights.append(h)
    heights.append(h)
plt.figure(figsize=(15,9))
plt.subplot(2,1,1)
makePlotCh(np.array(GA_avgFile)[:,:8],np.array(noGA_avgFile)[:,:8],"GA","No GA","Subcortical regions",avglim,1)
for i, a in zip(np.arange(2, 31, 4), np.arange(0,17,2)):
    if starstar[i]==True:
        barplot_annotate_brackets(a, a+1, '**', bars, heights, dh=0.1)
    else:
        if star[i]==True:
            barplot_annotate_brackets(a, a+1, '*', bars, heights, dh=0.1)
plt.subplot(2,1,2)
makePlotCh(np.array(GA_avgFile)[:,8:],np.array(noGA_avgFile)[:,8:],"GA","No GA","Cortical regions",avglim, leg=True)
for i, a in zip(np.arange(34, 66, 4), np.arange(0,17,2)):
    if starstar[i]==True:
        barplot_annotate_brackets(a, a+1, '**', bars, heights[16:], dh=0.1)
    else:
        if star[i]==True:
            barplot_annotate_brackets(a, a+1, '*', bars, heights[16:], dh=0.1)
plt.subplots_adjust(hspace=0.4, wspace=0.35)
plt.savefig(folder+'Plots/App_AvgRegions.png', dpi=200, bbox_inches='tight')
plt.show()

# motionfree time
heights_ = np.amax(noGA_motionfreeFile, axis=0)
heights=[]
for h in heights_:
    heights.append(h)
    heights.append(h)
plt.figure(figsize=(15,9))
plt.subplot(2,1,1)
makePlotCh(np.array(GA_motionfreeFile)[:,:8],np.array(noGA_motionfreeFile)[:,:8],"GA","No GA","Subcortical regions",mflim+15,1, lab_mf=True)
for i, a in zip(np.arange(3, 32, 4), np.arange(0,17,2)):
    if starstar[i]==True:
        barplot_annotate_brackets(a, a+1, '**', bars, heights, dh=0.1)
    else:
        if star[i]==True:
            barplot_annotate_brackets(a, a+1, '*', bars, heights, dh=0.1)
plt.subplot(2,1,2)
makePlotCh(np.array(GA_motionfreeFile)[:,8:],np.array(noGA_motionfreeFile)[:,8:],"GA","No GA","Cortical regions",mflim+15, lab_mf=True, leg=True)
for i, a in zip(np.arange(35, 67, 4), np.arange(0,17,2)):
    if starstar[i]==True:
        barplot_annotate_brackets(a, a+1, '**', bars, heights[16:], dh=0.1)
    else:
        if star[i]==True:
            barplot_annotate_brackets(a, a+1, '*', bars, heights[16:], dh=0.1)
plt.subplots_adjust(hspace=0.4, wspace=0.35)
plt.savefig(folder+'Plots/App_MotionfreeRegions.png', dpi=200, bbox_inches='tight')
plt.show()

# 7. Scores:

In [ ]:
score=[]
for pat in noGA_patients:
    print(pat)
    find = glob.glob('/mnt/mocodata1/BUF/DataMMR/'+pat+'/PETMR*/Score.csv')
    if len(find)>0:
        with open(find[0] ,'r') as mat:
            lines = mat.read().splitlines()
            for l in lines[1:]:
                print(l[-14:-13])
                score.append(l[-14:-13])
score = np.array(score, dtype=int)

In [ ]:
print(len(np.where(score==1)[0])/len(score))
print(len(np.where(score==2)[0])/len(score))
print(len(np.where(score==3)[0])/len(score))

print(len(np.where(score==1)[0]))
print(len(np.where(score==2)[0]))
print(len(score))

# 8. Plot the whole motion curves for centroid of GA and no GA patients:

In [ ]:
'''# extract the whole curves for GA patients:
allx = []
ally = []
allz = []
for sub in GA:
    centr = np.loadtxt(folder+'GA/centroids '+sub+'.out')
    filenames = os.listdir(folder+'GA/matrices '+sub)
    pcl_centr = PClCentroid(sub)
    x = []
    y = []
    z = []
    for file in filenames:
        mat = scipy.io.loadmat(folder+'GA/matrices '+sub+'/'+file)
        if "ref" not in file:
            transf = mat["matrix"]
            ec = np.append(pcl_centr,1)
            newCentroid = np.dot(transf, ec)
            x.append((ec[0]-newCentroid[0]))
            y.append((ec[1]-newCentroid[1]))
            z.append((ec[2]-newCentroid[2]))
        
    allx.append(x)
    ally.append(y)
    allz.append(z)
    
count = 0
for sub in GA:
    np.savetxt(folder+'GA axis/whole_xREF '+sub+'.out', np.array(allx[count]))
    np.savetxt(folder+'GA axis/whole_yREF '+sub+'.out', np.array(ally[count]))
    np.savetxt(folder+'GA axis/whole_zREF '+sub+'.out', np.array(allz[count]))
    count+=1'''

In [ ]:
'''# extract the whole curves for noGA patients:
allx = []
ally = []
allz = []
for sub in no_GA:
    centr = np.loadtxt(folder+'No GA/centroids '+sub+'.out')
    filenames = os.listdir(folder+'No GA/matrices '+sub)
    pcl_centr = PClCentroid(sub)
    x = []
    y = []
    z = []
    for file in filenames:
        mat = scipy.io.loadmat(folder+'No GA/matrices '+sub+'/'+file)
        if "ref" not in file:
            transf = mat["matrix"]
            ec = np.append(pcl_centr,1)
            newCentroid = np.dot(transf, ec)
            x.append((ec[0]-newCentroid[0]))
            y.append((ec[1]-newCentroid[1]))
            z.append((ec[2]-newCentroid[2]))
        
    allx.append(x)
    ally.append(y)
    allz.append(z)
    
count = 0
for sub in no_GA:
    np.savetxt(folder+'No GA axis/whole_xREF '+sub+'.out', np.array(allx[count]))
    np.savetxt(folder+'No GA axis/whole_yREF '+sub+'.out', np.array(ally[count]))
    np.savetxt(folder+'No GA axis/whole_zREF '+sub+'.out', np.array(allz[count]))
    count+=1'''

In [ ]:
noGA_medianx = []
noGA_mediany = []
noGA_medianz = []
noGA_allx = []
noGA_ally = []
noGA_allz = []
for sub in no_GA:
    xx = []
    yy = []
    zz = []
    print(sub)
    x = abs(np.loadtxt(folder+'No GA axis/whole_xREF '+sub+'.out'))
    y = abs(np.loadtxt(folder+'No GA axis/whole_yREF '+sub+'.out'))
    z = abs(np.loadtxt(folder+'No GA axis/whole_zREF '+sub+'.out'))
    change = False            
            
    if change == True:
        x = xx
        y = yy
        z = zz
        print("change")
    noGA_allx.append(x)
    noGA_ally.append(y)
    noGA_allz.append(z)
    mex = np.median(np.around(np.array(x), decimals=2)).tolist()
    mey = np.median(np.around(np.array(y), decimals=2)).tolist()
    mez = np.median(np.around(np.array(z), decimals=2)).tolist()

    noGA_medianx.append(mex)
    noGA_mediany.append(mey)
    noGA_medianz.append(mez)

In [ ]:
GA_medianx = []
GA_mediany = []
GA_medianz = []
GA_allx = []
GA_ally = []
GA_allz = []
a = 0
for sub in GA:
    xx = []
    yy = []
    zz = []
    print(sub)
    x = abs(np.loadtxt(folder+'GA axis/whole_xREF '+sub+'.out'))
    y = abs(np.loadtxt(folder+'GA axis/whole_yREF '+sub+'.out'))
    z = abs(np.loadtxt(folder+'GA axis/whole_zREF '+sub+'.out'))

    change = False

    if change == True:
        x = xx
        y = yy
        z = zz
        print("change")
    GA_allx.append(x)
    GA_ally.append(y)
    GA_allz.append(z)
    mex = np.median(np.around(np.array(x), decimals=2)).tolist()
    mey = np.median(np.around(np.array(y), decimals=2)).tolist()
    mez = np.median(np.around(np.array(z), decimals=2)).tolist()

    GA_medianx.append(mex)
    GA_mediany.append(mey)
    GA_medianz.append(mez)
    a+=1

In [ ]:
for sub in range(len(GA)):
    plt.plot(GA_allx[sub], 'tab:blue', label='x')
    plt.plot(GA_ally[sub], 'tab:green', label='y')
    plt.plot(GA_allz[sub], 'tab:orange', label='z')
    plt.legend()
    plt.title(GA[sub]+'  '+str(sub))
    plt.xlabel('Frame Number')
    plt.ylabel('Displacement [mm]')
    plt.show()

In [ ]:
for sub in range(len(no_GA)):
    plt.plot(noGA_allx[sub], 'tab:blue', label='x')
    plt.plot(noGA_ally[sub], 'tab:green', label='y')
    plt.plot(noGA_allz[sub], 'tab:orange', label='z')
    plt.legend()
    plt.title(no_GA[sub]+'  '+str(sub))
    plt.xlabel('Frame Number')
    plt.ylabel('Displacement [mm]')
    plt.show()

choose GA subject 11 2016_09_30_AMRH and no GA subject 31 2017_09_05_NSH for representative curve

In [ ]:
tmp, _, t1, t2 = StartEndFrameNr2('2016-09-30_AMRH', Scan_Times)
tmp, _, t3, t4 = StartEndFrameNr2('2017-02-14_SLA', Scan_Times)
print(t1, t2, t3, t4)


In [ ]:
x1_ = os.listdir(folder+'No GA/matrices 2017-02-14_SLA/')
x1=[]
for x in x1_:
    x1.append(int(x[2:-4]))
x2_ = os.listdir(folder+'GA/matrices 2016-09-30_AMRH/')
x2=[]
for x in x2_:
    x2.append(int(x[2:-4]))

In [ ]:
plt.figure(figsize=(12,7))
plt.subplot(2,1,1)
for a, b in zip(t3, t4):
    print(a,b)
    plt.axvspan(a, b, alpha=0.3, facecolor='tab:gray', edgecolor='white')
plt.plot(x1, noGA_allx[25], 'tab:blue', label='x')
plt.plot(x1, noGA_ally[25], 'tab:green', label='y')
plt.plot(x1, noGA_allz[25], 'tab:orange', label='z')
#plt.legend()
plt.title('Patient without GA')
plt.xlabel('Frame Number')
plt.ylabel('Displacement [mm]')
plt.subplot(2,1,2)
for a, b in zip(t1, t2):
    print(a,b)
    plt.axvspan(a, b, alpha=0.3, facecolor='tab:gray', edgecolor='white')
plt.plot(x2, GA_allx[11], 'tab:blue', label='x')
plt.plot(x2, GA_ally[11], 'tab:green', label='y')
plt.plot(x2, GA_allz[11], 'tab:orange', label='z')
plt.legend(bbox_to_anchor=(1.03,1.5), loc='upper left', borderaxespad=0.)
plt.title('Patient with GA')
plt.xlabel('Frame Number')
plt.ylabel('Displacement [mm]')
plt.ylim(0,8)
plt.subplots_adjust(hspace=0.6)
plt.savefig(folder+'Plots/Exempl_Motion.png', dpi=200, bbox_inches='tight')
plt.show()

# 9. Look at scan times:

In [ ]:
def StrToTime(float):
    return datetime.datetime.strptime(float, '%H%M%S.%f')

def SubstrTimes(time1, time2):
    diff = StrToTime(time1)-StrToTime(time2)
    return diff.total_seconds()

In [ ]:
# find wall times in minutes / beginning of first MR scan to end of last MR scan
wall_times = []
all_subj = [j for i in zip(GA, no_GA) for j in i]
for s in all_subj:
    lines = Scan_Times[Scan_Times[:,0]==s]
    lines = lines[:,2:].astype(float)
    minim = np.amin(lines)
    maxim = np.amax(lines)
    wall_times.append(SubstrTimes(str(maxim), str(minim))/60)

print(wall_times)
print('')
print('Wall times: Median +- std: ', np.median(wall_times), np.std(wall_times))

In [ ]:
# find total MR/PET protocol duration:
dur = []
for s in all_subj:
    sequ = glob.glob('/mnt/mocodata1/BUF/DataMMR/'+s+'/*/*')
    times = []
    for seq in sequ:
        if 'Score.csv' not in seq and 'Summary.csv' not in seq:
            #print(seq)
            dcm = os.listdir(seq+'/')[0] 
            try:
                times.append(getAcqTimeFromDICOM(seq+'/'+dcm))
            except Exception:
                pass
    times = np.array(times).astype(float)
    minim = np.amin(times)
    maxim = np.amax(times)
    dur.append(SubstrTimes(str(maxim), str(minim)))
    
print(dur)

In [ ]:
print(np.amin(dur), np.amax(dur))
print(np.amin(dur)/60, np.amax(dur)/60)
# exclude 2017-03-28_NPN because long pauses in between
test = np.append(dur[:-2], dur[-1])
print(np.amin(test), np.amax(test))
print(np.amin(test)/60, np.amax(test)/60)


# Analysis for excluded children:

In [ ]:
# get all excluded subjects:
subjs = ['2015-04-14_NJ', '2015-04-24_NJ', '2015-05-05_SB', '2015-05-08_ADS', '2015-05-26_PN', '2015-11-03_ELV', '2015-11-13_PL', '2016-01-05_JJW', '2016-02-26_FLTH', '2016-03-11_MBH', '2016-04-01_NLL', '2016-05-03_HF', '2016-05-03_MBH', '2016-05-24_RGR2', '2016-07-08_JK', '2016-08-02_JJ', '2016-10-19_BA', '2016-12-16_OKR', '2017-01-24_JD', '2017-02-24_HGSF', '2017-03-03_BA', '2017-04-21_ATWJ', '2017-05-05_AJ', '2017-05-19_FSH', '2017-08-22_SB', '2017-08-29_SEKB', '2017-09-01_MK', '2017-10-10_SB', '2017-11-03_VWN', '2017-11-28_ToM', '2015-11-10_BSA']
print(len(subjs))

In [ ]:
# go through GA list and sort subjs:
GA = ['2015-04-14_NJ', '2015-04-24_NJ',  '2015-05-05_SB',  '2016-07-08_JK',  '2016-10-19_BA',  '2017-02-24_HGSF', '2017-03-03_BA', '2017-04-21_ATWJ', '2017-05-05_AJ', '2017-09-01_MK', '2015-05-08_ADS']
no_GA = ['2015-11-10_BSA', '2017-11-28_ToM', '2015-05-26_PN', '2015-11-03_ELV', '2015-11-13_PL', '2016-02-26_FLTH', '2016-03-11_MBH',  '2016-04-01_NLL', '2016-05-03_MBH',  '2017-01-24_JD', '2017-05-19_FSH', '2017-08-22_SB', '2017-08-29_SEKB', '2017-10-10_SB', '2016-05-03_HF', '2016-08-02_JJ', '2017-11-03_VWN']

In [ ]:
Scan_Times = np.loadtxt(folder+'Scan_Times_Excl.txt', dtype=str, delimiter=', ', skiprows=1)

In [ ]:
''''''#save to file GA info / point cloud centroid added to centroids 
for sub in GA:
    print(sub)
    pcl_centr = PClCentroid(sub)
    print('PointCloud centroid: ', pcl_centr)
    c = [pcl_centr] #np.append(c_, [pcl_centr], axis=0)
    np.savetxt(folder+'GA_Excl/centroids '+sub+'.out', np.array(c))
    mat_files = StartEndFrameNr([sub], Scan_Times, folder+'GA_Excl/')
    h = computeHistograms2(mat_files,c)
    np.savetxt(folder+'GA_Excl/histogramREF '+sub+'.out', np.array(h))'''

In [ ]:
# load GA centroids and motion data ('histogram'):
GA_centr_PCl = []
GA_max_PCl = []
GA_avg_PCl = []
GA_median_PCl = []
GA_patients = []
GA_motionfree_PCl = []
idx = 0
idxs = []
hGA = []
for sub in GA:
    print(sub)
    c = np.loadtxt(folder+'GA_Excl/centroids '+sub+'.out')
    h = np.loadtxt(folder+'GA_Excl/histogramREF '+sub+'.out')
    h1  = []
    modified = False
    if not modified :
        h1 = h
        print("do not change")
    else:
        print("change")
    npH1 = np.array(h1)
    hGA.append(h1)
    ma = np.amax(npH1).tolist()
    me = np.median(np.around(npH1, decimals=2)).tolist()
    avg = np.average(npH1).tolist()
    mp = []
    for his in [h1]:
        mp.append(np.array(np.where( np.array(abs(his-his[0])) <2 )).shape[1]*100/len(his))
    
    print(mp)
    GA_motionfree_PCl.append(mp)
    GA_centr_PCl.append(c)
    GA_max_PCl.append(ma)
    GA_avg_PCl.append(avg)
    GA_median_PCl.append(me)  
    GA_patients.append(sub)
    

Look at all motion data to make sure that imported correctly:

In [ ]:
for data, pat in zip(hGA, GA_patients):
    plt.plot(data.T)
    plt.title(pat)
    plt.xlabel('Frame Number (not continuous)')
    plt.ylabel('Displacement from reference')
    plt.show()

Same analysis for no GA subjects:

In [ ]:
#save to file no GA info / point cloud centroid added to centroids 
'''for sub in no_GA:
    print(sub)
    pcl_centr = PClCentroid(sub)
    print('PointCloud centroid: ', pcl_centr)
    c = [pcl_centr] 
    np.savetxt(folder+'No GA_Excl/centroids '+sub+'.out', np.array(c))
    mat_files = StartEndFrameNr([sub], Scan_Times, folder+'No GA_Excl/')
    h = computeHistograms2(mat_files,c)
    np.savetxt(folder+'No GA_Excl/histogramREF '+sub+'.out', np.array(h))'''

In [ ]:
noGAcentr_PCl = []
noGA_max_PCl = []
noGA_avg_PCl = []
noGA_median_PCl = []
noGA_motionfree_PCl = []
noGA_patients = []
h_noGA = []
for sub in no_GA:
    print(sub)
    c = np.loadtxt(folder+'No GA_Excl/centroids '+sub+'.out')
    h = np.loadtxt(folder+'No GA_Excl/histogramREF '+sub+'.out')
    h1 = []
    npH = np.array(h)
    exclude = False
    count = 0            
            
    if count!=16:
        h1 = h
        print(count)
        print("dont change")
    else:
        print("change")
    h_noGA.append(h1)
    npH1 = np.array(h1)     
 
    ma = np.amax(npH1).tolist()
    me = np.median(np.around(npH1, decimals=2)).tolist()
    print(me)
    avg = np.average(npH1).tolist()
    mp = []
    for his in [h1]:
        mp.append(np.array(np.where( np.array(abs(his-his[0])) <2 )).shape[1]*100/len(his)) 
    
    print(mp)
    noGA_motionfree_PCl.append(mp)
    noGAcentr_PCl.append(c)
    noGA_max_PCl.append(ma)
    noGA_avg_PCl.append(avg)
    noGA_median_PCl.append(me)
    noGA_patients.append(sub)

In [ ]:
for data, pat in zip(h_noGA, noGA_patients):
    plt.plot(data.T)
    plt.title(pat)
    plt.xlabel('Frame Number (not continuous)')
    plt.ylabel('Displacement from reference')
    plt.show()

In [ ]:
print(np.amax(GA_avg_PCl), np.amax(GA_median_PCl), np.amax(GA_max_PCl), np.amin(GA_motionfree_PCl))
print(np.median(GA_avg_PCl), np.median(GA_median_PCl), np.median(GA_max_PCl), np.median(GA_motionfree_PCl))

## Compare metrics for point cloud centroid:

In [ ]:
metrics_GA = [GA_avg_PCl, GA_median_PCl, GA_max_PCl, [x[0] for x in GA_motionfree_PCl]]
metrics_noGA = [noGA_avg_PCl, noGA_median_PCl, noGA_max_PCl, [x[0] for x in noGA_motionfree_PCl]]
titles = ['Mean Displacement', 'Median Displacement', 'Maximum Displacement', 'Motion-free Time']
ylabels = ['Displacement [mm]', 'Displacement [mm]', 'Displacement [mm]', 'Motion-free time [%]']
colors = ['tab:blue', 'tab:orange']

plt.figure(figsize=(10,7))
plt.rcParams.update({'font.size': 14})
for i in range(0, 4):
    plt.subplot(2,2,i+1)
    plt.boxplot([metrics_noGA[i], metrics_GA[i]])
    for k, j in zip(range(1, 3), [metrics_noGA[i], metrics_GA[i]]):
        plt.plot(np.ones(len(j))*k, j, 'o', c=colors[k-1], ms=4)
    plt.title(titles[i])
    plt.xticks([1,2], ['No GA', 'GA'])
    plt.ylabel(ylabels[i])
plt.subplots_adjust(hspace=0.4, wspace=0.35)
plt.savefig(folder+'Plots/Metrics_PClCentroid_Excl.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
print('No GA:')
print('Med:', np.median(noGA_median_PCl), '+-', np.std(noGA_median_PCl))
print('Mean:', np.median(noGA_avg_PCl), '+-', np.std(noGA_avg_PCl))
print('Max:', np.median(noGA_max_PCl), '+-', np.std(noGA_max_PCl))
print('Motionfree:', np.median(noGA_motionfree_PCl), '+-', np.std(noGA_motionfree_PCl))
print('GA:')
print('Med:', np.median(GA_median_PCl), '+-', np.std(GA_median_PCl))
print('Mean:', np.median(GA_avg_PCl), '+-', np.std(GA_avg_PCl))
print('Max:', np.median(GA_max_PCl), '+-', np.std(GA_max_PCl))
print('Motionfree:', np.median(GA_motionfree_PCl), '+-', np.std(GA_motionfree_PCl))

In [ ]:
np.savetxt(folder+'metrics_GA_Excl', np.array(metrics_GA))
np.savetxt(folder+'metrics_noGA_Excl', np.array(metrics_noGA))

Comparison of metrics for included and excluded scans above